In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [4]:
url = 'https://kenpom.com/index.php?y=' + str(2021)
f = requests.get(url)
soup = BeautifulSoup(f.text)

In [3]:
def Kenpom(year):
    
    url = 'https://kenpom.com/index.php?y=' + str(year)
    f = requests.get(url)
    soup = BeautifulSoup(f.text)
    rating_table = soup.find_all('table', {'id': 'ratings-table'})
    thead = rating_table[0].find_all('thead')
    table = rating_table[0]
    for x in thead:
        table = str(table).replace(str(x), '')
    kenpom = pd.read_html(table)[0]
    kenpom.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Adjusted Efficience Margin', 'AdjustOffense', 'AdjustOffense_Rank', 'AdjustDefense', 'AdjustDefense_Rank',
                   'AdjustTempo', 'AdjustTempo_Rank', 'Luck', 'Luck Rank', 'StrengthOfSchedule', 'StrengthOfSchedule_Rank', 'Avg_opponenet_offense', 'Avg_opponenet_offense_rank',
                   'Avg_opponenet_defense', 'Avg_opponenet_defense', 'Non-Conference_SOS', 'Non-Conference_SOS_Rank']
    kenpom = kenpom.set_index('Team')
    return(kenpom)

In [9]:
season = Kenpom(2022)

In [16]:
season.head(1)

,Rank,Conference,W-L,Adjusted Efficience Margin,AdjustOffense,AdjustOffense_Rank,AdjustDefense,AdjustDefense_Rank,AdjustTempo,AdjustTempo_Rank,Luck,Luck Rank,StrengthOfSchedule,StrengthOfSchedule_Rank,Avg_opponenet_offense,Avg_opponenet_offense_rank,Avg_opponenet_defense,Avg_opponenet_defense,Non-Conference_SOS,Non-Conference_SOS_Rank
Team,,,,,,,,,,,,,,,,,,,,
Gonzaga,1,WCC,10-2,30.07,119.3,2,89.2,11,72.4,29,-0.058,282,-0.36,194,100.9,175,101.3,222,-0.36,191


In [43]:
def findteam(team):
    return(season.loc[team])

In [21]:
def teambyconference(conference):
    return(season[season['Conference'] == conference])

In [34]:
teambyconference('SEC')

,Rank,Conference,W-L,Adjusted Efficience Margin,AdjustOffense,AdjustOffense_Rank,AdjustDefense,AdjustDefense_Rank,AdjustTempo,AdjustTempo_Rank,Luck,Luck Rank,StrengthOfSchedule,StrengthOfSchedule_Rank,Avg_opponenet_offense,Avg_opponenet_offense_rank,Avg_opponenet_defense,Avg_opponenet_defense,Non-Conference_SOS,Non-Conference_SOS_Rank
Team,,,,,,,,,,,,,,,,,,,,
Tennessee,9,SEC,8-2,23.33,109.2,38,85.8,2,70.8,77,-0.036,234,-1.37,223,100.1,219,101.5,232,-1.37,219
LSU,10,SEC,11-0,23.10,107.9,55,84.8,1,70.9,70,0.066,77,0.51,169,101.3,150,100.8,190,0.51,163
Auburn,12,SEC,10-1,22.80,112.5,16,89.7,15,71.3,57,-0.024,222,4.22,70,103.0,72,98.8,74,4.22,67
Kentucky,13,SEC,8-2,22.18,113.9,12,91.7,35,68.3,184,-0.101,326,-5.56,316,98.7,285,104.2,338,-5.56,316
Alabama,20,SEC,9-3,20.89,114.5,8,93.6,53,72.0,37,-0.001,178,9.10,17,107.0,6,97.9,43,9.10,16
Florida,30,SEC,8-3,17.26,108.4,48,91.2,28,69.2,142,-0.044,255,0.45,172,99.9,228,99.4,113,0.45,169
Mississippi St.,40,SEC,9-3,15.15,109.9,30,94.7,66,65.0,326,-0.058,280,-1.01,210,101.5,139,102.5,276,-1.01,207
Arkansas,41,SEC,10-2,14.88,109.4,37,94.5,61,73.2,17,0.046,105,-4.07,288,99.6,247,103.6,325,-4.07,284
Texas A&M,63,SEC,9-2,11.23,104.6,98,93.4,52,67.5,222,0.020,145,-4.24,295,98.6,291,102.8,289,-4.24,290


In [38]:
home = 'Gonzaga'
away = 'North Alabama'

In [39]:
def gamepredict(home,away):
    home = season.loc[home]
    away = season.loc[away]
    HomeOff = home['AdjustOffense']*1.014
    HomeDef = home['AdjustDefense']*0.986
    AwayOff = away['AdjustOffense']*0.986
    AwayDef = away['AdjustDefense']*1.014
    HomeTemp = home['AdjustTempo']
    AwayTemp = away['AdjustTempo']
    AvgTemp = season.AdjustTempo.mean()
    AvgOff = season.AdjustOffense.mean()
    Pace = (HomeTemp * AwayTemp)/AvgTemp
    HomePPP = (HomeOff*AwayDef)/AvgOff
    AwayPPP = (AwayOff*HomeDef)/AvgOff
    HomeScore = round(((HomePPP*Pace)/100),0)
    AwayScore = round(((AwayPPP*Pace)/100),0)
    
    return(f'The is predicted to be {HomeScore} {home.name} to {AwayScore} {away.name}, that is a spread of {HomeScore-AwayScore} and total of {HomeScore+AwayScore}')

In [40]:
gamepredict(home,away)

'The is predicted to be 89.0 Gonzaga to 56.0 North Alabama, that is a spread of 33.0 and total of 145.0'

Basic descriptive statistics on the data set, as appropriate.

- The data is 8 files with Kenpom data from 2015 to 2022 for all 351 Division 1 Ncaa basketball teams. The information on each team is team's kenpom ranking, this is based off of the Adjusted Efficience Margin, the team's name, conferenece, Win-Loss, adjusted efficiency, adjusted offense, adjusted defense, adjusted tempo, luck score, strength of schedule for non-conference and entire schedule as well as ranks for each of the previous categories. The information also includes the oppenents average adjusted offense, defense, and ranks for each. A lot of this information is probably foriegn to someone who isn't a fan of basketball. Basically these are the basic advanced analytics used by basketball experts. 

A brief discussion of the challenges putting together the data set. 

- This data set was difficult to gather because Kenpom doesn't want their data to be scraped. I found this with a lot of sports data. There are tons of ways to get sports data if you are willing to pay a fee. However, without paying for sports data it is difficult to get. Kenpom has much more robust data if you pay for their membership. Kenpom does have an API but I would also have to pay for that.
- Once, I figured out how to decipher the first bits of HTML and get to the section with the dataframe I wanted, it wasn't a difficult scrape. I really enjoy pandas and found a great function called pd.read_html that turned a bunch of random HTML into a nice pandas dataframe. Before finding this command I spent a lot of time putting things in lists and it was taking a lot more time.


Hypotheses that could be tested or explored with this data set. Included specific analyses that could be performed wherever possible. 
- One really cool thing about being able to pull Kenpom data is that you can test future predictions. Kenpom is a dynamic data set. When a game is played the data-set is immediately updated. Without being able to pull the kenpom dataset at a given point, it would be impossible to have a set of games to use the data to predict. When you go back to the Kenpom dataset to input the independent variables the dependent variable you are wanting to test is already included in the dynamic dataset.
- One analysis I would like to do is see if before New Years basketball efficiency stats have any idication on how a team would do in March Madness. I would pull the Kenpom data set before January and wait until March Madness is over in April. Then I could run a regression and see if my model was accurate at all. My perdiction is it is fairly accurate. Last year Baylor won March Madness and they were the number 2 kenpom team going in January 2021.